Consider the sine function $f(x) = \sin(x)$ in the interval from 0 to 1.  We choose as approximants the polynomials of second degree: $\{a_0 + a_1 x + a_2 x^2 \}$.  To compute the values $[a_0, a_1, a_2]$ that minimize this problem, we form first a 3-by-3 matrix containing the pairwise _dot products_ (the integral of the product of two functions) of the basic functions $\{1, x, x^2\}$ in the given interval.  Due to the nature of this problem, we obtain a Hilbert matrix of order 3:

$$\begin{pmatrix}
\langle 1,1 \rangle & \langle 1, x \rangle & \langle 1, x^2 \rangle \\
\langle x,1 \rangle & \langle x, x \rangle & \langle x, x^2 \rangle \\
\langle x^2,1 \rangle & \langle x^2, x \rangle & \langle x^2, x^2 \rangle
\end{pmatrix} = \begin{pmatrix}
1 & 1/2 & 1/3 \\
1/2 & 1/3 & 1/4 \\
1/3 & 1/4 & 1/5
\end{pmatrix}
$$

The right-hand side of the system is the column vector with the dot product of the sine function with each basic function in the given interval:

$$\begin{pmatrix}
\langle \sin x, 1 \rangle \\
\langle \sin x, x \rangle \\
\langle \sin x, x^2 \rangle
\end{pmatrix} = \begin{pmatrix}
1 - \cos(1) \\
\sin(1) - \cos(1) \\
2\sin(1) + \cos(1) - 2
\end{pmatrix}
$$

We compute the coefficients and the corresponding approximation polynomial as follows:

In [ ]:
import numpy as np, scipy.linalg as spla, matplotlib.pyplot as plt
    
%matplotlib inline

A = spla.hilbert(3)
b = np.array([1-np.cos(1), np.sin(1)-np.cos(1), 2*np.sin(1)+np.cos(1)-2])
   
print spla.solve(A, b)

poly1 = np.poly1d(spla.solve(A, b)[::-1])
print poly1

In [ ]:
f = np.sin
x = np.linspace(0,1,100)
knots = np.linspace(0,1,7)[1:-1]
weights = np.ones_like(x)

from scipy.interpolate import LSQUnivariateSpline

approximant = LSQUnivariateSpline(x, f(x), knots, k=3, w = weights, bbox = [0, 1]) 

spla.norm(f(x) - approximant(x))

In [ ]:
approximant.get_residual()**(.5)

In [ ]:
def f(x, y): return np.sin(x) + np.sin(y)
t = np.linspace(-3, 3, 100)
domain = np.meshgrid(t, t)
X, Y = domain
Z = f(*domain)

X = X.ravel()
Y = Y.ravel()
Z = Z.ravel()

kx = np.linspace(-3,3,12)[1:-1]
ky = kx.copy()
weights = np.ones_like(Z);
   
from scipy.interpolate import LSQBivariateSpline

In [ ]:
approximant = LSQBivariateSpline(X, Y, Z, kx, kx, w = weights)

In [ ]:
approximant.get_residual()

In [ ]:
from scipy.optimize import leastsq

def error_function(a):
    return a[0] + a[1] * x + a[2] * x**2 - np.sin(x)

def jacobian(a):
    return np.array([np.ones(100), x, x**2])

coeffs, success = leastsq(error_function, np.zeros((3,)))

poly2 = np.poly1d(coeffs[::-1])
print poly2

coeffs, success = leastsq(error_function, np.zeros((3,)), Dfun = jacobian, col_deriv=True)
   
poly3 = np.poly1d(coeffs[::-1])
print poly3

map(lambda f: spla.norm(np.sin(x) - f(x)), [poly1, poly2, poly3])

In [ ]:
from scipy.optimize import curve_fit

def approximant(t, a, b, c):
    return a + b * t + c * t**2

curve_fit(approximant, x, np.sin(x), np.ones((3,)))

In [ ]:
def error_function(a):
    return (a[0] + a[1]*x) / (a[2] + a[3]*x) - np.tan(2*x)

def jacobian(a):
    numerator = a[0] + a[1]*x
    denominator = a[2] + a[3]*x
    return np.array( [ 1./denominator, x/denominator, 
                      -1.0*numerator/denominator**2,
                      -1.0*x*numerator/denominator**2 ])

In [ ]:
x1 = np.zeros((4,))
x2 = np.ones((4,))
x3 = np.array([1,0,0.78,-1])

coeffs, success = leastsq(error_function, x1)
numerator = np.poly1d(coeffs[1::-1])
denominator = np.poly1d(coeffs[:1:-1])
print numerator, denominator

coeffs, success = leastsq(error_function, x1, Dfun=jacobian, col_deriv=True)
numerator = np.poly1d(coeffs[1::-1])
denominator = np.poly1d(coeffs[:1:-1])
print numerator, denominator

In [ ]:
coeffs, success = leastsq(error_function, x2)
numerator = np.poly1d(coeffs[1::-1])
denominator = np.poly1d(coeffs[:1:-1])
print numerator, denominator
spla.norm(np.tan(2*x) - numerator(x) / denominator(x))

In [ ]:
coeffs, success = leastsq(error_function, x2, Dfun=jacobian, col_deriv=True)
numerator = np.poly1d(coeffs[1::-1])
denominator = np.poly1d(coeffs[:1:-1])
print numerator, denominator
spla.norm(np.tan(2*x) - numerator(x) / denominator(x))

In [ ]:
coeffs, success = leastsq(error_function, x3)
numerator = np.poly1d(coeffs[1::-1])
denominator = np.poly1d(coeffs[:1:-1])
print numerator, denominator
spla.norm(np.tan(2*x) - numerator(x) / denominator(x))

In [ ]:
coeffs, success = leastsq(error_function, x3, Dfun=jacobian, col_deriv=True)
numerator = np.poly1d(coeffs[1::-1])
denominator = np.poly1d(coeffs[:1:-1])
print numerator, denominator
spla.norm(np.tan(2*x) - numerator(x) / denominator(x))

In [ ]:
approximation_info = leastsq(error_function, x3, full_output=True)

coeffs = approximation_info[0]
print coeffs
    
message = approximation_info[-2]
print message
  
infodict = approximation_info[2]
print 'The algorithm performed {0:2d} iterations'.format(infodict['nfev'])

In [ ]:
>>> arr = np.array( [1, 1e20, 1, -1e20] * 1000, dtype=np.float64)
>>> arr.sum()          # The answer should be, of course, 2000

In [ ]:
>>> from math import fsum
>>> fsum(arr)

In [ ]:
def sphericalGaussian(x0, y0, h, v):
    return lambda x,y: h * np.exp(-0.5 * ((x-x0)**2+(y-y0)**2) / v)
   
domain = np.indices((32, 32))
values = np.random.randn(3,4)
values[:,:2] += np.random.randint(1, 32, size=(3, 2))
values[:,2] += np.random.randint(1, 64, size=3)
values[:,3] += np.random.randint(1, 16, size=3)
print values
   
img = np.random.randn(32,32)

for k in xrange(3):
    img += sphericalGaussian(*values[k])(*domain)

In [ ]:
from math import fsum

def error_function(a):
    a = a.reshape(3,4)
    cx = a[:,0]    # x-coords
    cy = a[:,1]    # y-coords
    H = a[:,2]     # heights
    V = a[:,3]     # variances
    guess = np.zeros_like(img)
    for i in xrange(guess.shape[0]):
        for j in xrange(guess.shape[1]):
            guess[i,j] = fsum(H*np.exp(-0.5*((i-cx)**2+(j-cy)**2)/V))
    return np.ravel(guess-img)

In [ ]:
x0 = np.vectorize(int)(values)
print x0

leastsq(error_function, x0)

In [ ]:
coeffs, success = _
coeffs = coeffs.reshape(3,4)

output = np.zeros_like(img)

for k in xrange(3):
    output += sphericalGaussian(*coeffs[k])(*domain)

plt.figure(figsize=(12,6))
plt.subplot(121)
plt.imshow(img)
plt.title('target image')
plt.subplot(122)
plt.imshow(output)
plt.title('computed approximation')
plt.show()